In [1]:
from pyspark.sql import SparkSession


from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import LogisticRegressionWithSGD, LogisticRegressionWithLBFGS
from pyspark.mllib.linalg import Vectors

In [2]:
#pyspark init
builder = SparkSession.Builder().appName('creditcard_mllib')
spark = builder.getOrCreate()

your 131072x1 screen size is bogus. expect trouble
25/04/04 21:51:22 WARN Utils: Your hostname, minhnhat resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/04 21:51:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/04 21:51:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/04 21:51:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
#Read input file
raw_df = spark.read.csv('creditcard.csv', header=True, inferSchema=True)

In [4]:
#Convert to rdd
feature_cols = [col for col in raw_df.columns if col != 'Class']

# Use the local variable inside the lambda
data_rdd = raw_df.rdd.map(lambda row: LabeledPoint(
    float(row['Class']),
    [float(row[col]) for col in feature_cols]
))

In [5]:
data_rdd.take(5)

[LabeledPoint(0.0, [0.0,-1.3598071336738,-0.0727811733098497,2.53634673796914,1.37815522427443,-0.338320769942518,0.462387777762292,0.239598554061257,0.0986979012610507,0.363786969611213,0.0907941719789316,-0.551599533260813,-0.617800855762348,-0.991389847235408,-0.311169353699879,1.46817697209427,-0.470400525259478,0.207971241929242,0.0257905801985591,0.403992960255733,0.251412098239705,-0.018306777944153,0.277837575558899,-0.110473910188767,0.0669280749146731,0.128539358273528,-0.189114843888824,0.133558376740387,-0.0210530534538215,149.62]),
 LabeledPoint(0.0, [0.0,1.19185711131486,0.26615071205963,0.16648011335321,0.448154078460911,0.0600176492822243,-0.0823608088155687,-0.0788029833323113,0.0851016549148104,-0.255425128109186,-0.166974414004614,1.61272666105479,1.06523531137287,0.48909501589608,-0.143772296441519,0.635558093258208,0.463917041022171,-0.114804663102346,-0.183361270123994,-0.145783041325259,-0.0690831352230203,-0.225775248033138,-0.638671952771851,0.101288021253234,-

In [6]:
#Split data into training and testing sets

###training set proportion parameter:
train_size = 0.8
###

train_rdd, test_rdd = data_rdd.randomSplit([train_size, 1 - train_size], seed=24) #Fixed with seed for reproductivity

In [7]:
model = LogisticRegressionWithLBFGS().train(train_rdd, intercept= True)


25/04/04 21:51:47 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [9]:
print("Coefficients: ", model.weights)
print("Intercept: ", model.intercept)

Coefficients:  [-4.173487749281254e-06,0.07662392512342757,0.042576005612505734,-0.0025974604118212427,0.7281178050791836,0.15481781595261299,-0.10603229684482642,-0.0665275004915603,-0.17021858994378383,-0.2106919810443441,-0.822488482392006,-0.0834677354600778,0.07709858201171306,-0.33478911111899773,-0.5247270553356113,-0.05392097014264278,-0.20783234798656486,-0.018626752685821,0.04524432022927201,0.11351828995971547,-0.4274756204148072,0.35243966077230926,0.5662010105598124,-0.10486905436540986,0.1517720983788643,6.030122681178059e-05,0.036503607853744624,-0.7694951651147063,-0.2443587814471927,0.00086856484774287]
Intercept:  -8.311975875216993


In [ ]:
#Evaluate the model on training data
predictions = train_rdd.map(lambda p: (model.predict(p.features), p.label))
accuracy = predictions.filter(lambda x: x[0] == x[1]).count() / float(train_rdd.count())

In [12]:
#Evaluate the model on test data
predictions = test_rdd.map(lambda p: (model.predict(p.features), p.label))
accuracy = predictions.filter(lambda x: x[0] == x[1]).count() / float(test_rdd.count())
print("Accuracy: ", accuracy)

Accuracy:  0.9992469747648985
